# RePlay recommender models comparison

We will show the main RePlay functionality and compare performance of RePlay models on well-known MovieLens dataset. If you have not used RePlay before, start with 01_replay_basics.ipynb which introduces base concepts and describe main classes and functionality.

### Dataset
We will compare RePlay models on __MovieLens 1m__. 

### Dataset preprocessing: 
Ratings greater than or equal to 3 are considered as positive interactions.

### Data split
Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics
Quality metrics used:__ndcg@k, hitrate@k, map@k, mrr@k__ for k = 1, 5, 10
Additional metrics used: __coverage@k__ and __surprisal@k__.

In [ ]:
! pip install rs-datasets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%config Completer.use_jedi = False

In [4]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [5]:
import logging
import time

from pyspark.sql import functions as sf

from replay.data import Dataset, FeatureHint, FeatureInfo, FeatureSchema, FeatureType
from replay.data.dataset_utils import DatasetLabelEncoder
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal, Experiment
from replay.models import (
    ALSWrap, 
    ItemKNN,
    SLIM, 
    PopRec,
    RandomRec,
    UCB,
    Wilson, 
    Word2VecRec,
)

from replay.utils.session_handler import State
from replay.splitters import TimeSplitter
from replay.utils import convert2spark, get_log_info
from rs_datasets import MovieLens

`State` object allows passing existing Spark session or create a new one, which will be used by the all RePlay modules.

To create session with custom parameters ``spark.driver.memory`` and ``spark.sql.shuffle.partitions`` use function `get_spark_session` from `session_handler` module.

In [ ]:
spark = State().session
spark

In [7]:
spark.sparkContext.setLogLevel('ERROR')

In [8]:
logger = logging.getLogger("replay")

In [9]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 5
SEED = 42

## 0. Preprocessing <a name='data-preparator'></a>

### 0.1 Data loading

In [10]:
data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


### 0.2. Dataset preparation

#### interactions preprocessing

- converting to spark dataframe
- separating positive and negative feedback interactions
- spliting data for train/test and optimization dataframes
- creating Dataset instances for experiments
- encoding datasets

In [11]:
interactions_spark = convert2spark(data.ratings)
interactions_spark.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|   1193|     5|978300760|
|      1|    661|     3|978302109|
|      1|    914|     3|978301968|
|      1|   3408|     4|978300275|
|      1|   2355|     5|978824291|
+-------+-------+------+---------+
only showing top 5 rows



In [12]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with rating = 1
only_positives_interactions = interactions_spark.filter(sf.col('rating') >= 3).withColumn('rating', sf.lit(1))
only_positives_interactions.count()

836478

In [13]:
# train/test split 
train_spl = TimeSplitter(
    time_threshold=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    query_column="user_id",
)
train, test = train_spl.split(only_positives_interactions)
print('train info:\n', get_log_info(train, user_col="user_id", item_col="item_id"))
print('test info:\n', get_log_info(test, user_col="user_id", item_col="item_id"))

train info:
 total lines: 669181, total users: 5397, total items: 3569


test info:
 total lines: 86542, total users: 1139, total items: 3279


In [14]:
train.is_cached

False

In [15]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

(535343, 24241)

In [16]:
opt_train.is_cached

False

In [17]:
# negative feedback will be used for Wilson and UCB models
only_negatives_log = interactions_spark.filter(sf.col('rating') < 3).withColumn('rating', sf.lit(0.))
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(train
              .withColumn('rating', sf.lit(1.))
              .union(only_negatives_log.filter(sf.col('timestamp') < test_start))
             )
pos_neg_train.cache()
pos_neg_train.count()

798993

In [18]:
feature_schema = FeatureSchema(
    [
        FeatureInfo(
            column="user_id",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.QUERY_ID,
        ),
        FeatureInfo(
            column="item_id",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.ITEM_ID,
        ),
        FeatureInfo(
            column="rating",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.RATING,
        ),
        FeatureInfo(
            column="timestamp",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.TIMESTAMP,
        ),
    ]
)

In [19]:
all_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=interactions_spark,
)

train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=train,
)

test_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=test,
)

train_neg_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=pos_neg_train,
)

opt_train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_train,
)

opt_val_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_val,
)

In [20]:
encoder = DatasetLabelEncoder()
encoder.fit(all_dataset)
train_dataset = encoder.transform(train_dataset)
train_neg_dataset = encoder.transform(train_neg_dataset)
test_dataset = encoder.transform(test_dataset)
opt_train_dataset = encoder.transform(opt_train_dataset)
opt_val_dataset = encoder.transform(opt_val_dataset)

# 1. Metrics definition

In [21]:
# experiment is used for metrics calculation
col_names = {
    "item_column": train_dataset.feature_schema.item_id_column,
    "query_column": train_dataset.feature_schema.query_id_column,
    "rating_column": train_dataset.feature_schema.interactions_rating_column,
}
e = Experiment(
    test_dataset,
    {
        MAP(**col_names): K,
        NDCG(**col_names): K,
        HitRate(**col_names): K_list_metrics,
        Coverage(train_dataset.interactions, **col_names): K,
        Surprisal(train_dataset.interactions, **col_names): K,
        MRR(**col_names): K,
    }
)

# 2. Models training

In [22]:
def fit_predict_add_res(name, model, experiment, train_dataset, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    logs = {'dataset': train_dataset}
    predict_params = {'k': K, 'queries': test_dataset.query_ids}
    
    if isinstance(model, (Wilson, UCB)):
        logs['dataset'] = train_neg_dataset
    
    predict_params.update(logs)

    model.fit(**logs)
    fit_time = time.time() - start_time

    pred=model.predict(**predict_params)
    pred.cache()
    pred.count()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name + suffix, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    experiment.results.loc[name + suffix, 'fit_time'] = fit_time
    experiment.results.loc[name + suffix, 'predict_time'] = predict_time
    experiment.results.loc[name + suffix, 'metric_time'] = metric_time
    experiment.results.loc[name + suffix, 'full_time'] = (fit_time + 
                                                          predict_time +
                                                          metric_time)
    pred.unpersist()
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))

In [23]:
def full_pipeline(models, experiment, train_dataset, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))
        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train_dataset, 
                                         opt_val_dataset, 
                                         param_borders=params, 
                                         k=K, 
                                         budget=budget)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            model.set_params(**best_params)
        
        logger.info(msg='{} fit_predict started'.format(name))
        fit_predict_add_res(name, model, experiment, train_dataset, suffix)
        # here we call protected attribute to get all parameters set during model initialization
        experiment.results.loc[name + suffix, 'params'] = str(model._init_args)

## 2.1. Non-personalized models

In [24]:
non_personalized_models = {
    'Popular': [PopRec(), 'no_opt'], 
    'Random (uniform)': [RandomRec(seed=SEED, distribution='uniform'), 'no_opt'], 
    'Random (popularity-based)': [RandomRec(seed=SEED, distribution='popular_based'), {"alpha": [-0.5, 100]}],
    'UCB': [UCB(exploration_coef=0.5), 'no_opt'],
    'Wilson': [Wilson(), 'no_opt'],
}

In [25]:
%%time
full_pipeline(non_personalized_models, e, train_dataset)

09-Nov-23 21:35:36, replay, INFO: Popular started
09-Nov-23 21:35:36, replay, INFO: Popular fit_predict started
09-Nov-23 21:35:57, replay, INFO: Random (uniform) started                      
09-Nov-23 21:35:57, replay, INFO: Random (uniform) fit_predict started


          NDCG@10    MRR@10  Coverage@10  fit_time
Popular  0.243711  0.390426     0.033903  2.980317


09-Nov-23 21:36:15, replay, INFO: Random (popularity-based) started             
09-Nov-23 21:36:15, replay, INFO: Random (popularity-based) optimization started
[I 2023-11-09 21:36:15,142] A new study created in memory with name: no-name-64efec96-8eef-47f0-98be-abff8cee6621


                   NDCG@10    MRR@10  Coverage@10  fit_time
Popular           0.243711  0.390426     0.033903  2.980317
Random (uniform)  0.025494  0.064895     0.955730  4.099224


/root/work/replay_gitlab/RePlay/replay/optimization/optuna_objective.py:75: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  res[param] = suggest_fn(param, low=low, high=high)
[I 2023-11-09 21:36:26,279] Trial 0 finished with value: 0.06526111153002567 and parameters: {'distribution': 'popular_based', 'alpha': 0.0}. Best is trial 0 with value: 0.06526111153002567.
[I 2023-11-09 21:36:37,526] Trial 1 finished with value: 0.06554383484218108 and parameters: {'distribution': 'popular_based', 'alpha': 3.2844959678224335}. Best is trial 1 with value: 0.06554383484218108.
[I 2023-11-09 21:36:48,670] Trial 2 finished with value: 0.05033412245362242 and parameters: {'distribution': 'popular_based', 'alpha': 86.40388699630243}. Best is trial 1 with value: 0.06554383484218108.
[I 2023-11-09 21:36:59,485] Trial 3 finished with value: 0.0651757689133112 and pa

                            NDCG@10    MRR@10  Coverage@10  fit_time
Popular                    0.243711  0.390426     0.033903  2.980317
Random (popularity-based)  0.071726  0.152698     0.667694  3.664674
Random (uniform)           0.025494  0.064895     0.955730  4.099224


09-Nov-23 21:37:44, replay, INFO: Wilson started                                
09-Nov-23 21:37:44, replay, INFO: Wilson fit_predict started


                            NDCG@10    MRR@10  Coverage@10  fit_time
Popular                    0.243711  0.390426     0.033903  2.980317
Random (popularity-based)  0.071726  0.152698     0.667694  3.664674
Random (uniform)           0.025494  0.064895     0.955730  4.099224
UCB                        0.000458  0.001024     0.003082  4.213878


                            NDCG@10    MRR@10  Coverage@10  fit_time
Popular                    0.243711  0.390426     0.033903  2.980317
Wilson                     0.092121  0.180976     0.017092  3.067655
Random (popularity-based)  0.071726  0.152698     0.667694  3.664674
Random (uniform)           0.025494  0.064895     0.955730  4.099224
UCB                        0.000458  0.001024     0.003082  4.213878
CPU times: user 1.79 s, sys: 584 ms, total: 2.38 s
Wall time: 2min 23s


In [26]:
e.results.sort_values('NDCG@10', ascending=False)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
Popular,0.033903,0.284460,0.530290,0.645303,0.154974,0.390426,0.243711,0.118354,2.980317,8.029443,9.599163,20.608923,"{'use_rating': False, 'add_cold_items': True, ..."
Wilson,0.017092,0.083406,0.345040,0.414399,0.044559,0.180976,0.092121,0.262190,3.067655,6.384841,6.025806,15.478302,"{'alpha': 0.05, 'add_cold_items': True, 'cold_..."
Random (popularity-based),0.667694,0.071115,0.260755,0.383670,0.030417,0.152698,0.071726,0.320167,3.664674,6.337403,7.028727,17.030804,"{'distribution': 'popular_based', 'alpha': 3.2..."
Random (uniform),0.955730,0.026339,0.118525,0.187884,0.008979,0.064895,0.025494,0.536993,4.099224,7.621786,5.893810,17.614820,"{'distribution': 'uniform', 'alpha': 0.0, 'see..."
UCB,0.003082,0.000000,0.000878,0.004390,0.000124,0.001024,0.000458,1.000000,4.213878,7.450834,5.564673,17.229385,"{'exploration_coef': 0.5, 'sample': False, 'se..."


UCB is developed for iterative learning and improve its quality over time as the exploration reduces.

In [27]:

e.results.to_csv('res_21_rel_1.csv')

## 2.2  Personalized models without features

In [28]:
common_models = {
  'Implicit ALS': [ALSWrap(seed=SEED), None], 
  'Explicit ALS': [ALSWrap(seed=SEED, implicit_prefs=False), None], 
  'ItemKNN': [ItemKNN(), None], 
  'SLIM': [SLIM(seed=SEED), None],
  'Word2Vec': [Word2VecRec(seed=SEED), None],
}

In [29]:
%%time
full_pipeline(common_models, e, train_dataset)

09-Nov-23 21:38:01, replay, INFO: Implicit ALS started
09-Nov-23 21:38:01, replay, INFO: Implicit ALS optimization started
[I 2023-11-09 21:38:01,080] A new study created in memory with name: no-name-534eabb3-a198-4124-91b6-93f92a3423cd
/root/anaconda3/envs/replay/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
[I 2023-11-09 21:38:18,873] Trial 0 finished with value: 0.2106789106218419 and parameters: {'rank': 10}. Best is trial 0 with value: 0.2106789106218419.
[I 2023-11-09 21:39:29,584] Trial 1 finished with value: 0.16951885280473594 and parameters: {'rank': 168}. Best is trial 0 with value: 0.2106789106218419.
[I 2023-11-09 21:39:44,957] Trial 2 finished with value: 0.20408667117350138 and parameters: {'rank': 16}. Best is trial 0 with value: 0.2106789106218419.
[I 2023-11-09 21:40:26,380] Trial 3 finished with value: 0.17588983611193507 and parameters: {'rank': 123}. Best 

                            NDCG@10    MRR@10  Coverage@10  fit_time
Implicit ALS               0.255083  0.412843     0.147380  4.232460
Popular                    0.243711  0.390426     0.033903  2.980317
Wilson                     0.092121  0.180976     0.017092  3.067655
Random (popularity-based)  0.071726  0.152698     0.667694  3.664674
Random (uniform)           0.025494  0.064895     0.955730  4.099224
UCB                        0.000458  0.001024     0.003082  4.213878


/root/anaconda3/envs/replay/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
[I 2023-11-09 21:41:30,374] Trial 0 finished with value: 0.01812132636739519 and parameters: {'rank': 10}. Best is trial 0 with value: 0.01812132636739519.
[I 2023-11-09 21:42:16,236] Trial 1 finished with value: 0.030450100352490097 and parameters: {'rank': 130}. Best is trial 1 with value: 0.030450100352490097.
[I 2023-11-09 21:42:35,897] Trial 2 finished with value: 0.021990602742316994 and parameters: {'rank': 57}. Best is trial 1 with value: 0.030450100352490097.
[I 2023-11-09 21:43:42,961] Trial 3 finished with value: 0.011212131208794445 and parameters: {'rank': 161}. Best is trial 1 with value: 0.030450100352490097.
[I 2023-11-09 21:43:58,616] Trial 4 finished with value: 0.021497462164507222 and parameters: {'rank': 27}. Best is trial 1 with value: 0.030450100352490097.
09-Nov-23 21:43:58, repla

                            NDCG@10    MRR@10  Coverage@10   fit_time
Implicit ALS               0.255083  0.412843     0.147380   4.232460
Popular                    0.243711  0.390426     0.033903   2.980317
Wilson                     0.092121  0.180976     0.017092   3.067655
Random (popularity-based)  0.071726  0.152698     0.667694   3.664674
Random (uniform)           0.025494  0.064895     0.955730   4.099224
Explicit ALS               0.007000  0.016119     0.229756  37.636191
UCB                        0.000458  0.001024     0.003082   4.213878


[I 2023-11-09 21:45:14,215] Trial 0 finished with value: 0.2081514555056189 and parameters: {'num_neighbours': 10, 'shrink': 0, 'weighting': None}. Best is trial 0 with value: 0.2081514555056189.
[I 2023-11-09 21:45:25,275] Trial 1 finished with value: 0.21195505392431113 and parameters: {'num_neighbours': 12, 'shrink': 42, 'weighting': 'tf_idf'}. Best is trial 1 with value: 0.21195505392431113.
[I 2023-11-09 21:45:35,108] Trial 2 finished with value: 0.1977977925447799 and parameters: {'num_neighbours': 2, 'shrink': 80, 'weighting': 'bm25'}. Best is trial 1 with value: 0.21195505392431113.
[I 2023-11-09 21:45:44,710] Trial 3 finished with value: 0.22835743698753844 and parameters: {'num_neighbours': 79, 'shrink': 21, 'weighting': 'bm25'}. Best is trial 3 with value: 0.22835743698753844.
[I 2023-11-09 21:45:53,447] Trial 4 finished with value: 0.2265011466254284 and parameters: {'num_neighbours': 53, 'shrink': 32, 'weighting': 'tf_idf'}. Best is trial 3 with value: 0.22835743698753844.

                            NDCG@10    MRR@10  Coverage@10   fit_time
Implicit ALS               0.255083  0.412843     0.147380   4.232460
ItemKNN                    0.252847  0.406924     0.084898  19.725513
Popular                    0.243711  0.390426     0.033903   2.980317
Wilson                     0.092121  0.180976     0.017092   3.067655
Random (popularity-based)  0.071726  0.152698     0.667694   3.664674
Random (uniform)           0.025494  0.064895     0.955730   4.099224
Explicit ALS               0.007000  0.016119     0.229756  37.636191
UCB                        0.000458  0.001024     0.003082   4.213878


/root/work/replay_gitlab/RePlay/replay/optimization/optuna_objective.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  res[param] = suggest_fn(param, low=low, high=high)
[I 2023-11-09 21:46:47,417] Trial 0 finished with value: 0.186897920331224 and parameters: {'beta': 0.01, 'lambda_': 0.01}. Best is trial 0 with value: 0.186897920331224.
[I 2023-11-09 21:47:10,504] Trial 1 finished with value: 0.17883298089540098 and parameters: {'beta': 2.6961430128807925e-05, 'lambda_': 5.261276060407379e-06}. Best is trial 0 with value: 0.186897920331224.
[I 2023-11-09 21:47:23,316] Trial 2 finished with value: 0.0 and parameters: {'beta': 1.3099074343030768, 'lambda_': 0.3766820132070009}. Best is trial 0 with value: 0.186897920331224.
[I 2023-11-09 21:47:41,820] Trial 3 finished with value: 0.18291482985514113 and parameters: {'beta': 

                            NDCG@10    MRR@10  Coverage@10   fit_time
Implicit ALS               0.255083  0.412843     0.147380   4.232460
ItemKNN                    0.252847  0.406924     0.084898  19.725513
Popular                    0.243711  0.390426     0.033903   2.980317
SLIM                       0.239433  0.414076     0.149902   9.945165
Wilson                     0.092121  0.180976     0.017092   3.067655
Random (popularity-based)  0.071726  0.152698     0.667694   3.664674
Random (uniform)           0.025494  0.064895     0.955730   4.099224
Explicit ALS               0.007000  0.016119     0.229756  37.636191
UCB                        0.000458  0.001024     0.003082   4.213878


/root/anaconda3/envs/replay/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
[I 2023-11-09 21:48:49,088] Trial 0 finished with value: 0.13023642300480168 and parameters: {'rank': 100, 'window_size': 1, 'use_idf': False}. Best is trial 0 with value: 0.13023642300480168.
[I 2023-11-09 21:50:13,935] Trial 1 finished with value: 0.030819348058484887 and parameters: {'rank': 187, 'window_size': 97, 'use_idf': True}. Best is trial 0 with value: 0.13023642300480168.
[I 2023-11-09 21:51:18,845] Trial 2 finished with value: 0.043864868562868524 and parameters: {'rank': 263, 'window_size': 36, 'use_idf': False}. Best is trial 0 with value: 0.13023642300480168.
[I 2023-11-09 21:52:22,698] Trial 3 finished with value: 0.03290632154835151 and parameters: {'rank': 119, 'window_size': 86, 'use_idf': True}. Best is trial 0 with value: 0.13023642300480168.
[I 2023-11-09 21:53:19,511] Trial 4 fini

                            NDCG@10    MRR@10  Coverage@10   fit_time
Implicit ALS               0.255083  0.412843     0.147380   4.232460
ItemKNN                    0.252847  0.406924     0.084898  19.725513
Popular                    0.243711  0.390426     0.033903   2.980317
SLIM                       0.239433  0.414076     0.149902   9.945165
Word2Vec                   0.138890  0.250942     0.154665   5.783883
Wilson                     0.092121  0.180976     0.017092   3.067655
Random (popularity-based)  0.071726  0.152698     0.667694   3.664674
Random (uniform)           0.025494  0.064895     0.955730   4.099224
Explicit ALS               0.007000  0.016119     0.229756  37.636191
UCB                        0.000458  0.001024     0.003082   4.213878
CPU times: user 9.74 s, sys: 2.45 s, total: 12.2 s
Wall time: 15min 56s


In [30]:
(
    e.results
    .sort_values('NDCG@10', ascending=False)
    [["Coverage@10", "HitRate@1", "HitRate@5", "HitRate@10", "MAP@10", "MRR@10", "NDCG@10", "Surprisal@10", "fit_time", "predict_time", "metric_time", "full_time"]]
)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time
Implicit ALS,0.147380,0.302019,0.566286,0.677788,0.159175,0.412843,0.255083,0.170293,4.232460,11.204253,5.849478,21.286191
ItemKNN,0.084898,0.298507,0.559263,0.658472,0.159895,0.406924,0.252847,0.146625,19.725513,9.108273,6.926844,35.760630
Popular,0.033903,0.284460,0.530290,0.645303,0.154974,0.390426,0.243711,0.118354,2.980317,8.029443,9.599163,20.608923
SLIM,0.149902,0.298507,0.569798,0.668130,0.141215,0.414076,0.239433,0.176244,9.945165,5.607941,5.082356,20.635462
Word2Vec,0.154665,0.151010,0.388060,0.494293,0.072467,0.250942,0.138890,0.245248,5.783883,25.519822,6.528785,37.832490
Wilson,0.017092,0.083406,0.345040,0.414399,0.044559,0.180976,0.092121,0.262190,3.067655,6.384841,6.025806,15.478302
Random (popularity-based),0.667694,0.071115,0.260755,0.383670,0.030417,0.152698,0.071726,0.320167,3.664674,6.337403,7.028727,17.030804
Random (uniform),0.955730,0.026339,0.118525,0.187884,0.008979,0.064895,0.025494,0.536993,4.099224,7.621786,5.893810,17.614820
Explicit ALS,0.229756,0.004390,0.032485,0.058824,0.001987,0.016119,0.007000,0.720019,37.636191,11.115317,5.679184,54.430692
UCB,0.003082,0.000000,0.000878,0.004390,0.000124,0.001024,0.000458,1.000000,4.213878,7.450834,5.564673,17.229385


In [31]:
e.results.to_csv('res_22_rel_1.csv')

# 3. Results

The best results by quality and time were shown by the commonly-used models such as ALS, ItemKNN, SLIM and Popular

In [32]:
e.results.sort_values('NDCG@10', ascending=False).head(5)

,Coverage@10,HitRate@1,HitRate@5,HitRate@10,MAP@10,MRR@10,NDCG@10,Surprisal@10,fit_time,predict_time,metric_time,full_time,params
Implicit ALS,0.147380,0.302019,0.566286,0.677788,0.159175,0.412843,0.255083,0.170293,4.232460,11.204253,5.849478,21.286191,"{'rank': 10, 'implicit_prefs': True, 'seed': 42}"
ItemKNN,0.084898,0.298507,0.559263,0.658472,0.159895,0.406924,0.252847,0.146625,19.725513,9.108273,6.926844,35.760630,"{'shrink': 21, 'use_rating': False, 'num_neigh..."
Popular,0.033903,0.284460,0.530290,0.645303,0.154974,0.390426,0.243711,0.118354,2.980317,8.029443,9.599163,20.608923,"{'use_rating': False, 'add_cold_items': True, ..."
SLIM,0.149902,0.298507,0.569798,0.668130,0.141215,0.414076,0.239433,0.176244,9.945165,5.607941,5.082356,20.635462,"{'beta': 0.01, 'lambda_': 0.01, 'seed': 42, 'i..."
Word2Vec,0.154665,0.151010,0.388060,0.494293,0.072467,0.250942,0.138890,0.245248,5.783883,25.519822,6.528785,37.832490,"{'rank': 100, 'window_size': 1, 'use_idf': Fal..."
